<a href="https://www.kaggle.com/code/wasjaip/auto-yandex-mast-v1?scriptVersionId=152774869" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Данные
train.csv - информация о характеристиках автомобилей (440000), которые будут использоваться в качестве обучающих данных.
test.csv - информация о характеристиках автомобилей (110000), которые будут использоваться в качестве тестовых данных.

Ваша задача - предсказать значение 'sellingprice' для каждого автомобиля из этого датасета.
sample_submission.csv - пример файла предсказаний в правильном формате.

sample_submission.csv - пример файла предсказаний в правильном формате.
vin - идентификатор автомобиля в тестовом наборе.
sellingprice - Целевой признак. Для каждого автомобиля предскажите числовое значение стоимости продажи.   Описание полей данных

year: Год выпуска автомобиля.
Make: Марка автомобиля.
Model: издание автомобиля определенной марки.
Trim: Уровни отделки салона автомобиля — это просто разные версии модели.
Body: Тип кузова транспортного средства относится к форме и модели конкретной марки автомобиля.
Transmission: механизм, который передает мощность от двигателя к колесам.
VIN: идентификационный номер транспортного средства.
State: штат.
Condition: Состояние автомобилей на момент аукциона.
Odometer: расстояние, пройденное автомобилем с момента выпуска.
Color: Цвет кузова автомобиля.
Interior: Цвет салона автомобиля.
Seller: Продавец автомобиля, автосалоны.
sellingprice: цена, по которой автомобиль был продан на аукционе.
saledate: Дата продажи автомобиля.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder


In [ ]:


# Загрузка
train_data = pd.read_csv('/kaggle/input/used-cars-price-prediction-22ds/train.csv')
test_data = pd.read_csv('/kaggle/input/used-cars-price-prediction-22ds/test.csv')
sample_submission = pd.read_csv('/kaggle/input/used-cars-price-prediction-22ds/sample_submission.csv')

train_data.head()


In [ ]:
test_data.head()


In [ ]:
train_data['state'].unique()

In [ ]:
sample_submission.head()

In [ ]:
# пустые
missing_train = train_data.isnull().sum()
missing_test = test_data.isnull().sum()

missing_train[missing_train > 0].sort_values(ascending=False)

### Имеются пропущенные значения в следующих столбцах как для обучающего, так и для тестового датасетов:

* Make (марка автомобиля)
* Model (модель автомобиля)
* rim (уровень отделки салона)
* Body (тип кузова)
* Condition (состояние автомобиля)
* Odometer (пробег автомобиля)
* Color (цвет кузова)
* Interior (цвет салона)

In [ ]:
# Замена пропусков на 0 во всех колонках, кроме 'transmission'
test_data = test_data.fillna({col: 0 for col in test_data.columns if col != "transmission"})

# Проверка процентного соотношения пропусков после замены
missing_percentage_cleaned = (test_data.isnull().sum() / len(test_data)) * 100
missing_percentage_cleaned



In [ ]:
len(train_data)

In [ ]:
# Список столбцов с пропущенными данными
columns_with_missing = ['make', 'model', 'trim', 'body', 'transmission', 'condition', 'odometer', 'color', 'interior']
# Находим строки, где все значения из списка пропущены
rows_to_drop = train_data[train_data[columns_with_missing].isnull().all(axis=1)].index

# Удаляем найденные строки из тренировочного датасета
train_data.drop(rows_to_drop, inplace=True)

# Возвращаем количество удаленных строк
len(rows_to_drop)


In [ ]:
len(train_data)

В тренировочном датасете нет строк, где все 9 указанных пунктов пропущены. Таким образом, мы не удалили ни одной строки.

### заполнить пропущенные значения в столбце "Model" на основе наиболее часто встречающейся модели для каждой марки автомобиля ("Make").!

In [ ]:
# Заполняем пропущенные значения в 'make' значением "Unknown"
train_data['make'].fillna('Unknown', inplace=True)

# Заполняем пропущенные значения в 'Model' на основе наиболее часто встречающейся модели для каждой марки
model_fill_values = train_data.groupby('make')['model'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown")
train_data['model'] = train_data.apply(lambda row: model_fill_values[row['make']] if pd.isnull(row['model']) else row['model'], axis=1)

# Проверяем количество пропущенных значений в столбце 'model' после заполнения
train_data['model'].isnull().sum()



проверка

In [ ]:
len(train_data)

Trim (уровень отделки салона) и Color (цвет кузова) и Interior (цвет салона) замени пустые на Unknown

In [ ]:
# Заполняем пропущенные значения в указанных столбцах значением "Unknown"
columns_to_fill = ['trim', 'color', 'interior']
for column in columns_to_fill:
    train_data[column].fillna('Unknown', inplace=True)
    # test_data[column].fillna('Unknown', inplace=True)
# Проверяем количество пропущенных значений в этих столбцах после заполнения
train_data[columns_to_fill].isnull().sum()
# test_data[columns_to_fill].isnull().sum()


In [ ]:
test_data[columns_to_fill].isnull().sum()

Остатки

In [ ]:
# Проверяем столбцы с пропущенными значениями в тренировочном датасете
remaining_missing = train_data.isnull().sum()
# Рассчитываем процент пропущенных значений для каждого столбца
missing_percentage = (remaining_missing / len(train_data)) * 100
missing_percentage.sort_values(ascending=False)

Процент пропущенных значений в указанных столбцах:

* Transmission (тип трансмиссии) - 11.69%
* Body (тип кузова) - 2.36%
* Condition (состояние автомобиля) - 2.14%
* Odometer (пробег автомобиля) - 0.016%

### Odometer удалить или сделать 0. данных минимум удалю. Второй варинт посмотреть состояние и год если 2015, поставить 0

In [ ]:
# Удаляем строки с пропущенными значениями в столбце 'odometer'
train_data.dropna(subset=['odometer'], inplace=True)
# Проверяем количество пропущенных значений в столбце 'odometer' после удаления
missing_odometer = train_data['odometer'].isnull().sum()
missing_odometer


In [ ]:
len(train_data)

### Condition (состояние автомобиля) - 2.14%   варианты

In [ ]:
# Показываем уникальные значения в столбце 'condition' и их количество
condition_values = train_data['condition'].value_counts(dropna=False)
condition_values.sort_values(ascending=False)


Цифры от 1.0 до 5.0 представляют состояние автомобиля, где 1.0 - наихудшее состояние, а 5.0 - наилучшее.
NaN представляет пропущенные значения, и их количество составляет 9356.

Заменим пропуски. ищем такаю же марку и модель у которой нет пропуска .
Для каждой строки с пропущенным значением в "Condition":
* Найдем такую же марку и модель без пропущенных значений в "Condition".
* Определим разброс цен для этой марки и модели и разделим его на 5 частей.
* В зависимости от значения "sellingprice" строки с пропущенным "Condition", назначим значение от 1 до 5.

In [ ]:
# Для оптимизации создадим словарь, который будет содержать информацию о разбросе цен для каждой комбинации Make и Model
price_ranges = train_data.groupby(['make', 'model'])['sellingprice'].agg(['min', 'max'])
price_ranges['range'] = (price_ranges['max'] - price_ranges['min']) / 5

def optimized_fill_condition(row):
    if pd.isnull(row['condition']):
        min_price, price_range = price_ranges.loc[(row['make'], row['model']), ['min', 'range']]

        # Если для данной комбинации Make и Model не найдено соответствующего разброса цен, возвращаем медианное значение "Condition"
        if pd.isnull(min_price) or pd.isnull(price_range):
            return train_data['condition'].median()

        # Возвращаем значение "Condition" на основе "sellingprice"
        if row['sellingprice'] <= min_price + price_range:
            return 1
        elif row['sellingprice'] <= min_price + 2 * price_range:
            return 2
        elif row['sellingprice'] <= min_price + 3 * price_range:
            return 3
        elif row['sellingprice'] <= min_price + 4 * price_range:
            return 4
        else:
            return 5
    else:
        return row['condition']

# Применяем оптимизированную функцию для заполнения пропущенных значений в "Condition"
train_data['condition'] = train_data.apply(optimized_fill_condition, axis=1)

# Проверяем количество пропущенных значений в столбце 'condition' после заполнения
missing_condition_optimized = train_data['condition'].isnull().sum()
missing_condition_optimized


Теперь в столбце "Condition" нет пропущенных значений.
Успешно оптимизировали процесс и заполнили пропуски, основываясь на комбинации "Make", "Model" и "sellingprice".

### Body (тип кузова)  тут  посмотрим на марку и модель  где нет  пропуска. модели и марки одного года выпуска имеют одинаковый кузов . заполняем пропуски

In [ ]:
# Проверяем количество пропущенных значений в столбце 'body' до заполнения
missing_body = train_data['body'].isnull().sum()
missing_body

In [ ]:
# Создаем словарь с типом кузова для каждой комбинации make, model и year
body_dict = train_data.dropna(subset=['body']).groupby(['make', 'model', 'year'])['body'].first().to_dict()

def fill_body(row):
    if pd.isnull(row['body']):
        return body_dict.get((row['make'], row['model'], row['year']))
    else:
        return row['body']

# Применяем функцию для заполнения пропущенных значений в столбце 'body'
train_data['body'] = train_data.apply(fill_body, axis=1)


# Проверяем количество пропущенных значений в столбце 'body' после заполнения
missing_body = train_data['body'].isnull().sum()
missing_body


После применения стратегии заполнения на основе марки, модели и года выпуска автомобиля, у нас осталось 10,370 пропущенных значений в столбце "Body".
 Это может быть из-за того, что для некоторых уникальных комбинаций марки, модели и года выпуска нет соответствующих записей в данных.

Уберем год. заполним  пропущенные значения в столбце "Body" на основе марки и модели автомобиля, не учитывая год выпуска.

In [ ]:
# Создаем словарь с типом кузова для каждой комбинации make и model
body_dict_without_year = train_data.dropna(subset=['body']).groupby(['make', 'model'])['body'].first().to_dict()

def fill_body_without_year(row):
    if pd.isnull(row['body']):
        return body_dict_without_year.get((row['make'], row['model']))
    else:
        return row['body']

# Применяем функцию для заполнения пропущенных значений в столбце 'body' без учета года
train_data['body'] = train_data.apply(fill_body_without_year, axis=1)

# Проверяем количество пропущенных значений в столбце 'body' после заполнения
missing_body_without_year = train_data['body'].isnull().sum()
missing_body_without_year


Body (тип кузова) замени пустые на   Unknown

In [ ]:
# Заполняем пропущенные значения в столбце 'body' значением "Unknown"
train_data['body'].fillna('Unknown', inplace=True)

# Проверяем количество пропущенных значений в столбце 'body' после заполнения
missing_body = train_data['body'].isnull().sum()
missing_body


Transmission (тип трансмиссии) тут  посмотрим на марку и модель  где нет  пропуска. модели и марки одного года выпуска имеют одинаковую трансмиссию . заполняем пропуски

In [ ]:
test_data.head()

In [ ]:
# Создаем словарь с типом трансмиссии для каждой комбинации make, model и year
transmission_dict = train_data.dropna(subset=['transmission']).groupby(['make', 'model', 'year'])['transmission'].first().to_dict()

def fill_transmission(row):
    if pd.isnull(row['transmission']):
        return transmission_dict.get((row['make'], row['model'], row['year']))
    else:
        return row['transmission']

# Применяем функцию для заполнения пропущенных значений в столбце 'transmission'
train_data['transmission'] = train_data.apply(fill_transmission, axis=1)
# Проверяем количество пропущенных значений в столбце 'transmission' после заполнения
missing_transmission = train_data['transmission'].isnull().sum()
missing_transmission


In [ ]:
train_data.head()
test_data.head()

После заполнения пропущенных значений в столбце "Transmission" на основе марки, модели и года выпуска автомобиля у нас осталось всего 182 пропущенных значения.

In [ ]:
# Создаем словарь с типом трансмиссии для каждой комбинации make, model и year
transmission_dict1 = test_data.dropna(subset=['transmission']).groupby(['make', 'model', 'year'])['transmission'].first().to_dict()

def fill_transmission(row):
    if pd.isnull(row['transmission']):
        return transmission_dict1.get((row['make'], row['model'], row['year']))
    else:
        return row['transmission']

# Применяем функцию для заполнения пропущенных значений в столбце 'transmission'
test_data['transmission'] = test_data.apply(fill_transmission, axis=1)
# Проверяем количество пропущенных значений в столбце 'transmission' после заполнения
missing_transmission = test_data['transmission'].isnull().sum()
missing_transmission

In [ ]:
# Замена пропусков на 0 во всех колонках, кроме 'transmission'
test_data['transmission']= test_data['transmission'].fillna(0)


In [ ]:
# Удаляем строки с пропущенными значениями в столбце 'transmission'
train_data.dropna(subset=['transmission'], inplace=True)

# Проверяем количество пропущенных значений в столбце 'transmission' после удаления
missing_transmission = train_data['transmission'].isnull().sum()
missing_transmission


In [ ]:
test_data.head()

In [ ]:
train_data.head()

In [ ]:
len(train_data)

In [ ]:
# Проверяем столбцы с пропущенными значениями в тренировочном датасете
remaining_missing = train_data.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]
remaining_missing


посмотрим на распределение  sellingprice

In [ ]:
# Визуализация распределения 'sellingprice'
plt.figure(figsize=(12, 6))
sns.histplot(train_data['sellingprice'], bins=1000, kde=True)
plt.title('Распределение цены продажи (sellingprice)')
plt.xlabel('Цена продажи')
plt.ylabel('Количество автомобилей')
plt.grid(True)
plt.show()


In [ ]:
# Визуализация ящика с усами для 'sellingprice'
plt.figure(figsize=(12, 6))
plt.boxplot(train_data['sellingprice'], vert=False)
plt.title('Ящик с усами для цены продажи (sellingprice)')
plt.xlabel('Цена продажи')
plt.grid(True)
plt.show()


На графике представлен "ящик с усами" для стоимости продажи (sellingprice).

* Медианное значение (центральная линия внутри ящика) показывает, что средняя цена продажи автомобилей находится в районе 10,000-15,000.
* Нижний и верхний квартили (нижняя и верхняя границы ящика) показывают, что большинство автомобилей продается по цене от примерно 5,000 до примерно 20,000.
* Усы графика показывают распределение значений вне этого интерквартильного диапазона. Значения за пределами усов можно рассматривать как выбросы. В нашем случае есть много автомобилей, которые продавались по цене выше обычного диапазона.
* Этот график подтверждает наше предыдущее наблюдение о том, что большинство автомобилей имеют цену в нижнем и среднем диапазоне, но также существует меньшинство автомобилей с высокой стоимостью.

годы выпуска , графически

In [ ]:
# Визуализация распределения года выпуска автомобилей
plt.figure(figsize=(14, 6))
sns.countplot(data=train_data, x='year', order=train_data['year'].value_counts().index)
plt.title('Распределение автомобилей по годам выпуска')
plt.xlabel('Год выпуска')
plt.ylabel('Количество автомобилей')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()


На графике представлено распределение автомобилей по годам выпуска.

* Можно заметить, что большинство автомобилей в датасете относятся к недавним годам выпуска, примерно начиная с 2000 года.
* Пик количества автомобилей приходится на 2015-2019 годы.
* Это может быть связано с тем, что автомобили более новых годов выпуска чаще продаются на вторичном рынке, чем старые модели.

In [ ]:
# Визуализация ящика с усами для года выпуска автомобилей
plt.figure(figsize=(12, 6))
plt.boxplot(train_data['year'], vert=False)
plt.title('Ящик с усами для года выпуска автомобилей')
plt.xlabel('Год выпуска')
plt.grid(True)
plt.show()


На графике представлен "ящик с усами" для года выпуска автомобилей:

* Медианное значение (центральная линия внутри ящика) указывает на то, что средний год выпуска автомобилей в датасете находится в районе 2016 года.*
* Нижний и верхний квартили (нижняя и верхняя границы ящика) показывают, что большинство автомобилей было выпущено между 2009 и 2019 годами.
* Усы графика показывают распределение значений вне этого интерквартильного диапазона. Значения за пределами усов можно рассматривать как выбросы. В нашем случае есть некоторое количество автомобилей, выпущенных до 2002 года.
* Этот график подтверждает наше предыдущее наблюдение о том, что большинство автомобилей в датасете относится к недавним годам выпуска.

создадим новый признак , raritet, и все авто меньше  1998 года поставим 1 , а 1998 и 2015 поставим 0

In [ ]:
test_data.head()

In [ ]:
test_data['raritet'] = test_data['year'].apply(lambda x: 1 if x < 1998 else (0 if 1998 <= x <= 2015 else None))

In [ ]:
# Создаем новый признак 'raritet'
train_data['raritet'] = train_data['year'].apply(lambda x: 1 if x < 1998 else (0 if 1998 <= x <= 2015 else None))
test_data['raritet'] = test_data['year'].apply(lambda x: 1 if x < 1998 else (0 if 1998 <= x <= 2015 else None))

# Проверяем первые строки датасета для нового признака
train_data[['year', 'raritet']].head()


Новый признак "raritet" :

Автомобили, выпущенные до 1998 года, получили значение 1.
Автомобили, выпущенные между 1998 и 2015 годами, получили значение 0.

In [ ]:
# Подсчитываем количество автомобилей с raritet = 1
raritet_count = train_data[train_data['raritet'] == 1].shape[0]
raritet_count

sellingprice сколько равно 0, сколько равно  меньше 1000, сколько больше 45000

In [ ]:
# Подсчитываем количество автомобилей с различными значениями 'sellingprice'
count_zero = train_data[train_data['sellingprice'] == 0].shape[0]
count_less_than_1000 = train_data[train_data['sellingprice'] < 1000].shape[0]
count_more_than_45000 = train_data[train_data['sellingprice'] > 45000].shape[0]

count_zero, count_less_than_1000, count_more_than_45000


распределение автомобилей по значениям 'sellingprice':

* Количество автомобилей с ценой продажи равной 0: 0 штук.
* Количество автомобилей с ценой продажи меньше 1000: 12,539 штук.
* Количество автомобилей с ценой продажи больше 45,000: 4,214 штук.

In [ ]:
# Определяем минимальную и максимальную даты продажи
min_saledate = train_data['saledate'].min()
max_saledate = train_data['saledate'].max()

min_saledate, max_saledate


In [ ]:

# Определяем минимальную и максимальную даты продажи в тестовом датасете
min_saledate_test = test_data['saledate'].min()
max_saledate_test = test_data['saledate'].max()

min_saledate_test, max_saledate_test


Даты продажи автомобилей в датасете охватывают следующий промежуток:

* Минимальная дата продажи: 3 апреля 2015 года
* Максимальная дата продажи: 27 мая 2015 года
* Таким образом, данные о продажах автомобилей охватывают период примерно в два месяца в 2015 году.

In [ ]:
train_data['saledate']

In [ ]:
# # Преобразование 'saledate' в строковый тип данных
# train_data['saledate1'] = train_data['saledate'].astype(str)
# test_data['saledate1'] = test_data['saledate'].astype(str)

# # Извлечение временной зоны и сохранение в новом столбце 'timezone'
# train_data['timezone'] = train_data['saledate1'].str.extract(r'(GMT[+-]\d{4})')
# test_data['timezone'] = test_data['saledate1'].str.extract(r'(GMT[+-]\d{4})')


In [ ]:
# Преобразование 'saledate' в строковый тип данных
train_data['saledate1'] = train_data['saledate'].astype(str)

# Извлечение временной зоны
train_data['timezone_str'] = train_data['saledate1'].str.extract(r'GMT([+-]\d{4})')

# Функция для преобразования временной зоны в число
def timezone_to_hours(tz_str):
    if pd.isna(tz_str):
        return None
    hours = (int(tz_str[1:3]) + int(tz_str[3:5])/60)*(-1)
    return hours if tz_str[0] == '+' else -hours

# Преобразование временной зоны в число
train_data['timezone'] = train_data['timezone_str'].apply(timezone_to_hours)

# Проверка результатов
print(train_data[['saledate', 'timezone']])


In [ ]:
# Преобразование 'saledate' в строковый тип данных
test_data['saledate1'] = test_data['saledate'].astype(str)

# Извлечение временной зоны
test_data['timezone_str'] = test_data['saledate1'].str.extract(r'GMT([+-]\d{4})')

# Функция для преобразования временной зоны в число
def timezone_to_hours(tz_str):
    if pd.isna(tz_str):
        return None
    hours = (int(tz_str[1:3]) + int(tz_str[3:5])/60)*(-1)
    return hours if tz_str[0] == '+' else -hours

# Преобразование временной зоны в число
test_data['timezone'] = test_data['timezone_str'].apply(timezone_to_hours)

# Проверка результатов
print(test_data[['saledate', 'timezone']])

In [ ]:
train_data['timezone']

In [ ]:
# Преобразование 'saledate' в UTC, затем в нужный формат
train_data['saledate'] = pd.to_datetime(train_data['saledate'], utc=True).dt.tz_convert(None)
train_data['saledate'] = train_data['saledate'].dt.strftime('%Y-%m-%d %H:%M')

test_data['saledate'] = pd.to_datetime(test_data['saledate'], utc=True).dt.tz_convert(None)
test_data['saledate'] = test_data['saledate'].dt.strftime('%Y-%m-%d %H:%M')



# Проверка результатов
train_data['saledate'].head()



In [ ]:
# Проверка результатов
test_data['saledate'].head()

In [ ]:
train_data['saledate'].dtype

In [ ]:

# Конвертация в datetime без временной зоны
train_data['saledate'] = pd.to_datetime(train_data['saledate'], errors='coerce')

# Конвертация года выпуска в datetime, если вам это нужно и 'year' - это название столбца
# Здесь предполагается, что 'year' уже хранит целочисленный год
train_data['release_date'] = pd.to_datetime(train_data['year'].astype(str), format='%Y', errors='coerce')



In [ ]:
# Проверка типов данных после конвертации
train_data['saledate'].dtype,train_data['release_date'].dtype

In [ ]:
# Для test_data


# Конвертация в datetime без временной зоны
test_data['saledate'] = pd.to_datetime(test_data['saledate'], errors='coerce')

# Конвертация года выпуска в datetime, если вам это нужно и 'year' - это название столбца
# Здесь предполагается, что 'year' уже хранит целочисленный год
test_data['release_date'] = pd.to_datetime(test_data['year'].astype(str), format='%Y', errors='coerce')

# Проверка типов данных после конвертации
test_data['saledate'].dtype,test_data['release_date'].dtype


In [ ]:
# Проверка на недопустимые значения
print(train_data['year'].unique())
print(test_data['year'].unique())


In [ ]:
train_data['saledate']

In [ ]:
test_data['saledate']

In [ ]:
# Вычисление 'days_in_use'
train_data['days_in_use'] = (train_data['saledate'] - pd.to_datetime(train_data['year'], format='%Y')).dt.days
test_data['days_in_use'] = (test_data['saledate'] - pd.to_datetime(test_data['year'], format='%Y')).dt.days

train_data[['saledate', 'timezone', 'days_in_use']].head()




In [ ]:
train_data.head()

In [ ]:
test_data[['saledate', 'timezone', 'days_in_use']].head()

In [ ]:
# Удаление столбца 'saledate'
train_data = train_data.drop(columns=['saledate','release_date','saledate1'])
test_data = test_data.drop(columns=['saledate','release_date','saledate1'])


train_data['timezone'].head(), test_data['timezone'].head()



все обработали давай построим модель для предсказания стоимости продажи автомобиля на вторичном рынке.
задача - предсказать значение 'sellingprice' для каждого автомобиля из этого датасета. test.csv,
на тренировочной выборке train.csv, вывод: sample_submission.csv - пример файла предсказаний в правильном формате.
vin - идентификатор автомобиля в тестовом наборе.
sellingprice - Целевой признак. Для каждого автомобиля предскажите числовое значение стоимости продажи.

* Удаление неинформативных столбцов и столбцов, которые не будут использоваться в модели (например, дата продажи, VIN и т. д.).
* Преобразование категориальных переменных в числовые с использованием метода кодирования (например, one-hot encoding).
* Разделение данных на признаки (X) и целевую переменную (y).

# РАСЧЕТ

In [ ]:
from sklearn.preprocessing import StandardScaler

# Удаляем ненужные столбцы
X_cluster = train_data.drop(columns=['sellingprice'])

# Определение категориальных признаков
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()

# Создание трансформера для категориальных признаков
categorical_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Создание итогового трансформера
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# Применяем препроцессор и одновременно обучаем его
X_preprocessed = preprocessor.fit_transform(X_cluster)

# Масштабирование признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_preprocessed)



In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Используем метод "локтя" для определения оптимального количества кластеров
inertia = []
for k in range(1, 50):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(X_scaled)
    inertia.append(kmeans.inertia_)

# График изменения инерции от количества кластеров
plt.figure(figsize=(10, 6))
plt.plot(range(1, 50), inertia, marker='o', linestyle='--')
plt.title('Изменение инерции от количества кластеров')
plt.xlabel('Количество кластеров')
plt.ylabel('Инерция')
plt.show()


In [ ]:
# Выберите оптимальное количество кластеров на основе графика
optimal_clusters = 50  # например, 4

# Применяем KMeans
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42).fit(X_scaled)

# Добавляем результаты кластеризации к нашему датафрейму
train_data['cluster'] = kmeans.labels_

In [ ]:
# Преобразование тестовых данных с использованием уже обученного препроцессора и масштабировщика
X_test_cluster = test_data  # Удаляем столбец 'saledate', так как вы говорили, что его больше нет
X_test_preprocessed = preprocessor.transform(X_test_cluster)
X_test_scaled = scaler.transform(X_test_preprocessed)

# Применение модели KMeans к преобразованным тестовым данным
test_data['cluster'] = kmeans.predict(X_test_scaled)


In [ ]:
test_data['cluster']

In [ ]:
# 1. Удаление неинформативных столбцов
#cols_to_drop = ['vin', 'saledate', 'sellingprice']
cols_to_drop = ['vin', 'sellingprice']
X_train = train_data.drop(columns=cols_to_drop)
y_train = train_data['sellingprice']


In [ ]:
X_test = test_data

In [ ]:
# Функция для расчета MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mape_scorer = make_scorer(mape, greater_is_better=False)


# Определение категориальных признаков
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
# Создание трансформера для категориальных признаков
categorical_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Создание итогового трансформера
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)


In [ ]:
X_train

In [ ]:
X_train.dtypes


In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

# Параметры для GridSearch
param_grid = {
    'model__iterations': [2000],
    'model__depth': [16],
    'model__learning_rate': [0.1],
    'model__l2_leaf_reg': [1]
}

# Создание конвейера с CatBoost
pipeline_catboost = Pipeline(steps=[('preprocessor', preprocessor),
                                    ('model', CatBoostRegressor( verbose=0, random_seed=42))
                                    ])

# # Использование GridSearchCV для поиска лучших параметров
# grid_search_catboost = GridSearchCV(pipeline_catboost, param_grid, cv=3, scoring=mape_scorer, verbose=2, n_jobs=-1)
# grid_search_catboost.fit(X_train, y_train)

grid_search_catboost = GridSearchCV(pipeline_catboost, param_grid, cv=3, scoring=mape_scorer, verbose=2, n_jobs=-1, error_score='raise')
grid_search_catboost.fit(X_train, y_train)

In [ ]:
# Получение лучшей модели из GridSearchCV
best_model = grid_search_catboost.best_estimator_['model']

# Получение важности признаков
feature_importances = best_model.get_feature_importance()

# Сопоставление имен признаков и их важности
features = X_train.columns
importance_df = pd.DataFrame({'features': features, 'importance': feature_importances})

# Сортировка признаков по важности и отображение на графике
importance_df = importance_df.sort_values(by='importance', ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(data=importance_df, x='importance', y='features')
plt.title("Важность признаков")
plt.xlabel("Важность")
plt.ylabel("Признаки")
plt.show()


In [ ]:


# Лучшие параметры
best_params = grid_search_catboost.best_params_

# Лучший пайплайн
best_pipeline_catboost = grid_search_catboost.best_estimator_

# Предсказание на тренировочной выборке
train_preds = best_pipeline_catboost.predict(X_train)

# Расчет MAPE
train_mape = mape(y_train, train_preds)

best_params, train_mape


In [ ]:
# Предсказание на тестовой выборке
test_preds = best_pipeline_catboost.predict(test_data)

# Запись результатов в файл
submission = pd.DataFrame({'vin': test_data['vin'], 'sellingprice': test_preds})
submission.to_csv('sample_submission.csv', index=False)


In [ ]:
train_mape